### AreTomo Alignment

In [ ]:
aretomo_md_file = f"{proj_name}_aretomo_mdout.yaml"

with open(aretomo_md_file, "r") as f:
    md = yaml.load(f.read(), Loader=yaml.FullLoader)

ts_list = md["process_list"]
stat_files = md["aretomo_align_stats"]

In [ ]:
def get_px_size():
    """ Returns aligned images pixel size in nm """

    mc_yaml_file = f"{proj_name}_mc2.yaml"
    if os.path.exists(mc_yaml_file) is False:
        raise FileNotFoundError("MotionCor2 yaml file not found, pixel size cannot be extracted")
    with open(mc_yaml_file, "r") as f:
        params_mc = yaml.load(f.read(), Loader=yaml.FullLoader)

    aretomo_align_yaml_file = f"{proj_name}_aretomo_align.yaml"
    if os.path.exists(aretomo_align_yaml_file) is False:
        raise FileNotFoundError("AreTomo align yaml file not found, pixel size cannot be extracted")
    with open(aretomo_align_yaml_file, "r") as f:
        params_at = yaml.load(f.read(), Loader=yaml.FullLoader)
    
    px_size = params_mc["MC2"]["desired_pixel_size"] * params_at["AreTomo_setup"]["output_binning"] * 0.1
    return float(px_size)

def get_stats(file_in):
    with open(file_in, "r") as f:
        lines = f.readlines()
        
    header = lines[2].strip().split()[1:]
    df = pd.read_csv(file_in, sep="\s+", comment="#", names=header)
    df["Abs_shift"] = np.sqrt(df.TX**2 + df.TY**2)
    
    return np.mean(df.Abs_shift), np.std(df.Abs_shift)

In [ ]:
mean_list = []
std_list = []

for idx, curr_file in enumerate(stat_files.values()):
    if os.path.isfile(curr_file) is False:
        print(f"AreTomo align: {curr_file} not found - skipping")
    else:
        mean, std = get_stats(curr_file)
        mean_list.append(mean)
        std_list.append(std)

if len(mean_list) > 0:    
    at_df = pd.DataFrame(columns=["Tilt series", "Mean shift (px)", "Shift s.d. (px)"])
    at_df["Tilt series"] = ts_list
    at_df["Mean shift (px)"] = mean_list
    at_df["Shift s.d. (px)"] = std_list

    px_size = get_px_size()
    at_df["Mean shift (A)"] = np.array(mean_list) * px_size
    at_df["Shift s.d. (A)"] = np.array(std_list)  * px_size

    at_df

else:
    print("AreTomo align stats file not found")

In [ ]:
if len(mean_list) > 0: 
    plt.figure(figsize=(10,5))
    ax = sns.barplot(data=at_df,
                    x="Tilt series",
                    y="Mean shift (A)",
                    color="cornflowerblue"
                    )

    ax.errorbar(x=at_df.index-1,
                y=at_df["Mean shift (A)"],
                yerr=at_df["Shift s.d. (A)"], fmt="none", c="k")

    plt.gca().set_ylim(bottom=0)
